In [23]:
import pandas as pd

In [24]:
from pandas import DataFrame

In [25]:
df_tennis = pd.read_csv("lab3.csv")
df_tennis

,Outlook,Temperature,Humidity,Windy,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rainy,Mild,High,Weak,Yes
4,Rainy,Cool,Normal,Weak,Yes
5,Rainy,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rainy,Mild,Normal,Weak,Yes


In [26]:
def entropy(probs):
    import math
    return sum([-prob * math.log(prob,2) for prob in probs])

In [33]:
def entropy_of_list(a_list):
    from collections import Counter
    cnt = Counter(x for x in a_list)
    num_instances = len(a_list)*1.0
    probs = [x/num_instances for x in cnt.values()]
    return entropy(probs)

In [34]:
total_entropy = entropy_of_list(df_tennis['PlayTennis'])
total_entropy

0.9402859586706309

In [29]:
def information_gain(df,split_attribute_name,target_attribute_name,trace=0):
    df_split = df.groupby(split_attribute_name)
    nobs = len(df.index)*1.0
    df_agg_ent = df_split.agg({target_attribute_name:[entropy_of_list, lambda x:len(x)/nobs]})[target_attribute_name]
    df_agg_ent.columns = ['Entropy','PropObservation']
    new_entropy = sum(df_agg_ent['Entropy']*df_agg_ent['PropObservation'])
    old_entropy = entropy_of_list(df[target_attribute_name])
    return old_entropy-new_entropy
    

In [45]:
def id3(df,target_attribute_name,atrribute_names,default_class=None):
    from collections import Counter
    cnt = Counter(x for x in df[target_attribute_name])
    if len(cnt) == 1:
        return next(iter(cnt))
    elif df.empty or (not attribute_names):
        return default_class
    else:
        default_class = max(cnt.keys())
        gainz = [information_gain(df,attr,target_attribute_name) for attr in attribute_names]
        index_of_max = gainz.index(max(gainz))
        best_attr = attribute_names[index_of_max]
        tree = {best_attr:{}}
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset, target_attribute_name, remaining_attribute_names,default_class)
            tree[best_attr][attr_val] = subtree
        return tree
        
    

In [46]:
attribute_names = list(df_tennis.columns)
attribute_names.remove('PlayTennis')
attribute_names

['Outlook', 'Temperature', 'Humidity', 'Windy']

In [47]:
from pprint import pprint
tree = id3(df_tennis,'PlayTennis',attribute_names)
print("\n The Resultant Decision Tree is: \n" )
pprint(tree)


 The Resultant Decision Tree is: 

{'Outlook': {'Overcast': 'Yes',
             'Rainy': {'Windy': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}
